# Read In URL 
## Description  
Program to input all the URL and read in the content 


In [9]:
import pandas as pd
path = "data/RSS_News_Feeds/"
file_name = path + "articles.xlsx"

In [10]:
xl_file = pd.ExcelFile(file_name)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [13]:
dfs.keys()

dict_keys(['Business Risk Management', 'GVCs', 'FDI', 'Biotechnology', 'Emerging Markets', 'Bioeconomy', 'Productivity and Sustainability', 'Market Development', 'Antimicrobial Resistance', 'Coronavirus', 'Food Safety', 'Livestock ', 'Domestic Support', 'Market Access', 'Innovation ', 'Regional Trade Agreements', 'Digital Opportunities', 'Taxation', 'Climate and Nat Res Const', 'Regulations-Regulatory Coop', 'Food Security ', 'Environmental Impacts-water', 'Water Risks', 'Legislation and Regulations', 'Healthy Eating', 'Food Consumption', 'Food Processing', 'Retail Food'])

In [26]:
counts = [len(dfs[k]) for k in dfs.keys()]
df_cnts = pd.DataFrame({"subject": list(dfs.keys()), "cnt": counts})

In [27]:
filtered_subjects = df_cnts[df_cnts.cnt > 9]
filtered_subjects

,subject,cnt
3,Biotechnology,29
4,Emerging Markets,32
5,Bioeconomy,10
6,Productivity and Sustainability,21
7,Market Development,15
9,Coronavirus,97
10,Food Safety,10
11,Livestock,10
13,Market Access,10
14,Innovation,22


## Extract URL

In [175]:
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import time

class w_driver():
    
    def __enter__(self):
        self.driver = webdriver.Firefox()
        return self.driver

    def __exit__(self, exc_type, exc_value, traceback):
        if self.driver is not None:
            self.driver.quit()
        self.driver = None
    
def web_content(mydriver, url):
    assert (len(url) > 5), "Invalid url"
    assert (url is not None), "Invalid url"
    mydriver.get(url)
#    time.sleep(1)
    html = mydriver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',
        'div',
        'meta'
        # there may be more elements you don't want, such as "style", etc.
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t.rstrip())

    return output        

In [204]:
for subject in filtered_subjects.subject:
    print(dfs[subject].columns)

Index(['TOPIC', 'TITLE', 'ISSUE/TREND', 'SOURCE/LINK', 'DATE PUBLISHED',
       'DATE RECORDED', 'LEAD', 'Recorded by', 'Sharepoint link',
       'RESEARCH ON THIS TOPIC'],
      dtype='object')
Index(['TOPIC', 'TITLE', 'ISSUE/TREND', 'SOURCE/LINK', 'DATE PUBLISHED',
       'DATE RECORDED', 'LEAD -HUGH', 'Recorded by', 'RESEARCH ON THIS TOPIC'],
      dtype='object')
Index(['TOPIC', 'TITLE', 'ISSUE/TREND', 'SOURCE/LINK', 'DATE PUBLISHED',
       'DATE RECORDED', 'LEAD', 'RESEARCH ON THIS TOPIC'],
      dtype='object')
Index(['TOPIC', 'TITLE', 'ISSUE/TREND', 'SOURCE/LINK', 'DATE PUBLISHED',
       'DATE RECORDED', 'LEAD', 'Recorded by', 'RESEARCH ON THIS TOPIC'],
      dtype='object')
Index(['TITLE', 'Topic/Title', 'ISSUE/TREND', 'SOURCE/LINK', 'DATE PUBLISHED',
       'DATE RECORDED', '(LEAD-Sudarma )           Contributed By',
       'RESEARCH ON THIS TOPIC'],
      dtype='object')
Index(['TITLE', 'ISSUE/TREND', 'Unnamed: 2', 'SOURCE/LINK', 'DATE PUBLISHED',
       'DATE RECORDED', 'L

In [205]:
inp = dfs[filtered_subjects.subject.iloc[1]]["SOURCE/LINK"][2]

l_topic = []
l_title = []
l_issue = []
l_source = []
l_pub_date = []
l_rec_date = []
l_content = []

with w_driver() as mydriver:
    for f_subject in filtered_subjects.subject:
        print(f_subject)
        for src in dfs[f_subject]['SOURCE/LINK']:
            content = ""; 
            try:
                addr = re.findall(r'(http:\/.*|https:\/.*|www\..*)$', src)[0].rstrip().lstrip()
                if addr[:4] != "http": addr = "http://" + addr
                try:
                    content = web_content(mydriver, addr)
                except:
                    pass
            except:
                pass
            l_content.append(content)
            
        l_topic = l_topic + list(dfs[f_subject]['TOPIC'])
        l_title = l_title + list(dfs[f_subject]['TITLE'])
        l_issue = l_issue + list(dfs[f_subject]['TITLE'])
        l_source = l_source + list(dfs[f_subject]['SOURCE/LINK'])
        l_pub_date = l_pub_date + list(dfs[f_subject]['DATE PUBLISHED'])
        l_rec_date = l_rec_date + list(dfs[f_subject]['DATE RECORDED'])
        
df = pd.DataFrame({"TOPIC":l_topic, "TITLE":l_title, "ISSUE":l_issue, "SOURCE":l_source, "PUB_DATE": l_pub_date, "REC_DATE": l_rec_date, "TEXT": l_content})        
df

Biotechnology


,TOPIC,TITLE,ISSUE,SOURCE,PUB_DATE,REC_DATE,TEXT
0,Biotechnology,Trending News on Crop Biotech in 2017,Trending News on Crop Biotech in 2017,ISAAA http://isaaablog.blogspot.ca/2018/01/tre...,2018-01-16 00:00:00,2018-01-22,ISAAA.ORG Blog: Trending News on Crop Biotech ...
1,Biotechnology,New plant breeding techniques should be exempt...,New plant breeding techniques should be exempt...,www.feednavigator.com/Article/2018/01/18/New-p...,2018-01-18 00:00:00,2018-02-09,New plant breeding techniques should be exempt...
2,Biotechnology- Industry Trend and Developments,Asia GM Outlook: Slow but steady move towards ...,Asia GM Outlook: Slow but steady move towards ...,Business Monitor International. Asia GM outlook,2018-02-13 00:00:00,2018-02-14,
3,Biotechnology- Industry Trend and Developments,Europe GM Outlook,Europe GM Outlook,https://collab.agr.gc.ca/co/int_an-an-int/Docu...,2018-03-12 00:00:00,2018-03-14,
4,Biotechnology,Global Agricultural Biotechnology Market Resea...,Global Agricultural Biotechnology Market Resea...,www.marketresearchstore.com/report/global-agri...,2017-02-01 00:00:00,2018-04-13,Global Agricultural Biotechnology Market Resea...
5,Biotechnology,Agricultural Biotechnology Annual-Canada (2017),Agricultural Biotechnology Annual-Canada (2017),USDA-GAIN Report CA17042,01/29/2018,2018-04-26,
6,Biotechnology,"At WTO, US spars with European Union over GE a...","At WTO, US spars with European Union over GE a...",https://iegpolicy.agribusinessintelligence.inf...,2018-06-01 00:00:00,2018-06-01,"At WTO, US spars with European Union over GE a..."
7,Biotechnology,Commission adopts ingredients origin labelling...,Commission adopts ingredients origin labelling...,https://iegpolicy.agribusinessintelligence.inf...,2018-06-01 00:00:00,2018-06-01,Commission adopts ingredients origin labelling...
8,Biotechnology,Global Status of Commercialized Biotech/GM Cro...,Global Status of Commercialized Biotech/GM Cro...,www.isaaa.org/resources/publications/briefs/53...,2018-06-26 00:00:00,2018-07-30,Global Status of Commercialized Biotech/GM Cro...
9,Biotechnology,OECD Conference on Genome Editing: Application...,OECD Conference on Genome Editing: Application...,www.oecd.org/environment/genome-editing-agricu...,"Online Accessed July 5, 2018",2018-07-05,Genome Editing: Applications in Agriculture - ...
